### Video Loading Tests

This file is for testing my methods for loading in the video data and manipulating it so that is ready for analysis/training

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 # OpenCV
import os

import Settings # My global settings
import Preprocessing # Work with the raw data

In [5]:
Preprocessing.testData()

Searching for data in location specified by Settings.DATA_LOCATION:
/run/media/jack/Seagate Portable Drive/Research/geogran2/
...
ERROR: Path /run/media/jack/Seagate Portable Drive/Research/geogran2/ does not exist


False

In [3]:
dataFiles = Preprocessing.listDataFiles(excludeDataWithoutMovies=False)
matFile = Preprocessing.loadMatFile(dataFiles[0])
#video = Preprocessing.loadVideo([dataFiles[0], dataFiles[7], dataFiles[18]], ignoreInvalidData=True)

#propertyFile = Preprocessing.loadRunData([dataFiles[20], dataFiles[1]])
#print(matFile["t"][:,0])
#print(matFile)
#print(propertyFile)

In [6]:
print(video)

[None, <VideoCapture 0x7f23afbe8df0>, <VideoCapture 0x7f23afbe8eb0>]


In [31]:
keys = []
values = []
with open('070525d.rundata') as file:
    for line in file.read().split('\n'):
        try:
            newKey, newValue = line.split('=')
            keys.append(newKey.strip())
            values.append(newValue.strip())
        except:
            pass # Ignore whitespace lines
    
runProperties = dict(zip(keys, values))

# Convert the good area to be a proper array
runProperties["goodarea"] = np.array([i.strip() for i in runProperties["goodarea"][1:-2].split()], dtype='int')

print(runProperties)

{'name': '070525d', 'velocity': '0.05', 'shearspringname': 'B', 'normalspringname': 'C2+2', 'numpart': 'N', 'boundary': '', 'fps': '4', 'pixpermm': '1.80', 'type': "{'GR'}", 'goodarea': array([ 79, 746, 100, 497]), 'centerline': '272', 'sync': '2.5', 'starttime': '5', 'goodtime': '120', 'endtime': '413', 'maskout': '[10:30]'}


In [48]:
video = cv2.VideoCapture('070525d.bw.avi')

if not video.isOpened():
    print('Error opening file')

    
ret, frame = video.read()
grayscaleFrame = frame[:,:,0]

reg = runProperties["goodarea"]

cv2.imshow('Frame', grayscaleFrame[reg[0]:reg[1], reg[2]:reg[3]])
cv2.imshow('Frame', grayscaleFrame)
cv2.waitKey(50000)

# while video.isOpened():
#     ret, frame = video.read()

#     if ret:
#         cv2.imshow('Frame', frame)
#         cv2.waitKey(25)
        
#     else:
#         break
        
video.release()

cv2.destroyAllWindows()